In [1]:
import json
import logging

from AgentFactory import AgentFactory
from Questionnaire import BigFIve

logging.basicConfig(level=logging.INFO)

agent_factory = AgentFactory()

/home/youyuan/miniconda3/envs/transcreation/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# config_path: str = "configs/conf_LLaVA.json"
config_path: str = "configs/conf_LLaMA3.json"
# config_path: str = "configs/conf_DeepSeek.json"
config = json.load(open(config_path, "r"))
config = config["model"]

agent = agent_factory.create_agent(config)

/home/youyuan/miniconda3/envs/transcreation/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Loading checkpoint shards: 100%|██████████| 4/4 [03:38<00:00, 54.67s/it] 


In [ ]:
agent.reset_msg_history()

agent.instruction = config["system_msg"]

prompt = [
        {"type": "text", "text": "Talks a lot\n"},
]

In [ ]:
response, text = agent.get_response(prompt)
# print(text)

### Loading a multi-modal model

In [ ]:
# config_path: str = "configs/conf_Gemini.json"
# config_path: str = "configs/conf_Qwen2.5.json"
config_path: str = "configs/conf_GPT.json"
config = json.load(open(config_path, "r"))
config = config["model"]

agent = agent_factory.create_agent(config)
agent.instruction = config["system_msg"]

In [ ]:
# agent.reset_msg_history()

prompt = [
            {"type": "text", "text": "Are the two images same?\n"},
            {"type": "text", "text": "return a json object with the following keys:\n"},
            {"type": "text", "text": "same: true/false\n"},
            {"type": "text", "text": "reason: string\n"},
            {"type": "text", "text": "confidence: 0-1\n"},
            {"type": "text", "text": "\n"},
            {"type": "text", "text": "Please answer in json format."},
            # {"type": "text", "text": "What are the differences between the two images?"},
            {"type": "image"},
            {"type": "image"},
        ]
images = ["assets/chatbot.png", "assets/folder.png"]

response, text = agent.get_response(prompt, images = images)

In [ ]:
response.usage

In [ ]:
print(response)
print(text)
agent.llm.total_cost()